<a href="https://colab.research.google.com/github/UncleTeslim/diabetes_prompt/blob/main/prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install set-env-colab-kaggle-dotenv -q
!pip install weave -U -q
!pip install litellm -U -q
# !pip install text_formatting -U -q

In [2]:
%%capture
# from text_formatting import render
from set_env import set_env
set_env("WANDB_API_KEY")
set_env("OPENAI_API_KEY")
set_env("GEMINI_API_KEY")

In [4]:
# from google.colab import userdata
# userdata.get('OPEN_AI_API')
# print("OPENAI_API_KEY")

OPENAI_API_KEY


In [3]:
%%capture
import weave
import litellm
completion = litellm.completion

In [4]:
OPENAI_MODEL_1 = "gpt-4o"
OPENAI_MODEL_2 = "gpt-4o-mini"
GOOGLE_MODEL_1 = "gemini-2.0-flash"
GOOGLE_MODEL_2 = "gemini-2.5-pro-exp-03-25"


DIABETES = "diabetes-Copy.md"

MODEL_NAME = GOOGLE_MODEL_2
PROMPT_GUIDE = DIABETES

In [5]:
weave.init("llm_self_project")

Logged in as Weights & Biases user: uncleteslim.
View Weave data at https://wandb.ai/uncleteslim-none/llm_self_project/weave


In [6]:
@weave.op()
def get_completion(system_message: str, messages: list, model: str, max_tokens: int = 4096, temperature: float = 0, custom_llm_provider: str = None, **kwargs) -> dict:
    """
    Generates a completion using the specified model, taking into account the system message, conversation history, and additional arguments.

    Parameters:
        system_message (str): A message providing context or instructions for the model.
        messages (list): A list of dictionaries representing the conversation history, where each dictionary has keys 'role' and 'content'.
        model (str): The identifier of the model to use for generating completions.
        max_tokens (int, optional): The maximum number of tokens to generate in the completion. Defaults to 4096.
        temperature (float, optional): The sampling temperature to control the randomness of the generated text. Defaults to 0.
        **kwargs: A dictionary of additional keyword arguments. Expected keys include 'system_message', 'model', 'max_tokens', and 'temperature'.

    Returns:
        dict: A dictionary representing the generated completion as JSON.
    """
    # Adjust messages format based on the model type
    if "gpt" in model.lower():
        formatted_messages = [{"role": "system", "content": system_message}] + messages
    else:
        formatted_messages = messages
        kwargs["system"] = system_message  # For non-gpt models, use system_message directly in kwargs

    # Common arguments for the completion function
    completion_args = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": formatted_messages,
        "custom_llm_provider": custom_llm_provider,
        **kwargs
    }

    # Generate and return the completion
    response = completion(**completion_args)
    return response.json()

In [7]:
@weave.op()
def prompt_llm(question: str, **kwargs) -> str:
    """
    Sends a question to the language model and returns its response.

    This function prepares a message with the user's question, handles additional
    arguments for the language model, and invokes the get_completion function to
    obtain a response. The response's content is then returned.

    Parameters:
        question (str): The question intended for the language model.
        **kwargs: A dictionary of additional keyword arguments. Expected keys include 'system_message', 'model', 'max_tokens', and 'temperature'.

    Returns:
        str: The language model's response to the question.
    """
    # Prepare the user's question for the language model
    messages = [{"role": "user", "content": question}]

    # Extract additional parameters, applying defaults if necessary
    system_message = kwargs.pop('system_message', "")
    model = kwargs.pop('model', MODEL_NAME)
    max_tokens = kwargs.pop('max_tokens', 4096)
    temperature = kwargs.pop('temperature', 0)
    custom_llm_provider = kwargs.pop('custom_llm_provider', None)

    # Compile arguments for the completion request
    completion_args = {
        "system_message": system_message,
        "messages": messages,
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "custom_llm_provider": custom_llm_provider
    }
    completion_args.update(kwargs)  # Properly include any other additional arguments

    # Request a completion from the language model
    response = get_completion(**completion_args)

    # Extract and return the content of the model's response
    return response["choices"][0]["message"]["content"]

In [10]:
# raw_prompt_response = prompt_llm(
#     "What is diabetes and what are the most common symptoms"
# )
system_message = """

"""
response = get_completion(
    system_message=system_message,
    messages=[{"role": "user", "content": "what is diabetes? i was just diagnosed with diabetes, what do i do?"}],
    model=MODEL_NAME,  # or any other model
    custom_llm_provider="gemini"  # specify the provider
)

🍩 https://wandb.ai/uncleteslim-none/llm_self_project/r/call/0196010d-bdf9-7141-af80-06268b390b66


In [11]:
# print(raw_prompt_response)
content = response['choices'][0]['message']['content']
print(content)

Okay, first things first: take a deep breath. Hearing you have diabetes can feel overwhelming, confusing, and maybe even scary. That's completely normal. But know this: **diabetes is manageable**, and you absolutely can live a long, healthy, and fulfilling life with it.

Let's break down your questions:

**1. What is Diabetes?**

In simple terms, diabetes is a chronic (long-lasting) health condition that affects how your body turns food into energy.

*   **The Process:** When you eat food, especially carbohydrates, your body breaks it down into sugar (glucose) which enters your bloodstream.
*   **The Hormone:** Your pancreas releases a hormone called **insulin**. Insulin acts like a key, allowing the glucose from your blood to enter your body's cells to be used for energy.
*   **What Goes Wrong in Diabetes:**
    *   **Type 1 Diabetes:** Your body's immune system mistakenly attacks and destroys the insulin-producing cells in your pancreas. This means your body makes very little or no i

In [13]:
def load_markdown_file(file_path: str) -> str:
    """
    Reads and returns the content of a markdown file specified by its path.

    Parameters:
        file_path (str): The path to the markdown file to be read.

    Returns:
        str: The content of the markdown file as a string.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_content = file.read()
    return markdown_content
context = load_markdown_file(PROMPT_GUIDE)

In [15]:
# context_prompt_response = prompt_llm(
#     context + "\n\nWhat is diabetes and what are the most common symptoms"
# )


context_prompt_response = prompt_llm(
    context + "\n\n68 year old male with type 2 diabetes and high blood pressure. What should this person be considering to manage their diabetes",
    model=MODEL_NAME,  # Specify the model you are using
    custom_llm_provider="gemini"  # Specify the provider for Gemini
)

🍩 https://wandb.ai/uncleteslim-none/llm_self_project/r/call/0196010f-9831-7e02-9529-7c9091d790f7


In [16]:
print(context_prompt_response)

Okay, based on the provided context about diabetes management, here's a breakdown of what a 68-year-old male with Type 2 Diabetes and High Blood Pressure should be considering:

**Key Focus Areas:**

1.  **The Diabetes ABCS - Especially B (Blood Pressure):**
    *   **A1C:** Understand his personalized A1C goal (often <7%, but might be adjusted based on age/health) and ensure regular testing (every 3-6 months typically). Discuss this goal specifically with his healthcare team.
    *   **Blood Pressure:** This is critical given his existing high blood pressure. He needs to work closely with his doctor to achieve and maintain his target BP (often below 130/80 mm Hg, potentially lower). This involves medication adherence and lifestyle changes. Controlling BP is vital to protect his heart, kidneys, brain, and eyes.
    *   **Cholesterol:** Regular lipid panel testing is essential. He needs to discuss his target cholesterol levels with his doctor and whether medication (like a statin) is ne

In [19]:
system_message = """
You are a knowledgeable assistant specializing in diabetes-related information.
Your role is to provide accurate and comprehensive answers to user inquiries about diabetes,
including its definition, symptoms, management, and treatment options.
You have access to a diabetes documentation resource that contains detailed information on various aspects of the condition.
Always ensure that your responses are clear, concise, and informative, and when necessary, reference the relevant sections of the documentation to support your answers.
If a user asks a question that requires specific data or insights from the documentation, retrieve that information and present it in a user-friendly manner.
"""

system_message_response = prompt_llm(
    context + "\n\na 29 year old married woman has been diagnosed with diabetes. shes yet to have a baby and is worried this might affect. how would you advise the person??",
    system_message=system_message,
    model=MODEL_NAME,
    custom_llm_provider="gemini"
)


🍩 https://wandb.ai/uncleteslim-none/llm_self_project/r/call/01960111-acca-75e0-a946-3cb81e0305eb


In [20]:
print(system_message_response)

Okay, this is a very common and understandable concern for a young woman newly diagnosed with diabetes. It's great that she's thinking ahead. Here's how you can advise her, drawing from the provided context and general knowledge about diabetes and pregnancy:

**1. Acknowledge and Validate Her Feelings:**
"It's completely understandable that you're worried about how diabetes might affect having a baby. A new diagnosis brings many questions, and thinking about pregnancy adds another layer. Please know that many women with diabetes have healthy pregnancies and healthy babies, but it does require careful planning and management."

**2. Emphasize the Importance of Pre-Conception Planning:**
"The most crucial step is to plan *before* you start trying to conceive. Getting your diabetes under the best possible control *before* pregnancy significantly reduces risks for both you and the baby."

**3. Connect to the Diabetes ABCS (Pre-Pregnancy Focus):**
"Think of the 'Diabetes ABCS' we discussed 

In [21]:
system_message = """
You are a knowledgeable assistant specializing in diabetes-related information.
Your role is to provide accurate and comprehensive answers to user inquiries about diabetes,
including its definition, symptoms, management, and treatment options.
You have access to a diabetes documentation resource that contains detailed information on various aspects of the condition.
Always ensure that your responses are clear, concise, and informative, and when necessary, reference the relevant sections of the documentation to support your answers.
If a user asks a question that requires specific data or insights from the documentation, retrieve that information and present it in a user-friendly manner.
"""

system_message_response = prompt_llm(
    context + "\n\n65 year old woman who just been diagnosed with diabetes. what sort of food should be be esating or avoiding??",
    system_message=system_message,
    model=MODEL_NAME,
    custom_llm_provider="gemini"
)

🍩 https://wandb.ai/uncleteslim-none/llm_self_project/r/call/01960116-806f-7030-a886-ef9bb5179e74


In [22]:
print(system_message_response)

Okay, based on the provided context about diabetes management, here's a guide on food choices for a newly diagnosed 65-year-old woman.

**Most Important First Step:**

*   **Consult the Healthcare Team:** The context strongly emphasizes personalization. The *best* advice will come from her doctor, a **registered dietitian (RD)**, or a **certified diabetes care and education specialist (CDCES)**. They can create a meal plan tailored to her specific needs, preferences, other health conditions, and medication.

**General Principles (Based on the Context):**

1.  **Focus on Balance & Portion Control, Not Strict Elimination:** As the context states, it's often about managing *how much* and *how often* you eat certain foods, rather than banning favorites entirely.
2.  **Manage the ABCS:** Food choices directly impact Blood Glucose (A1C), Blood Pressure, and Cholesterol. The goal is to choose foods that help manage all three.
3.  **Consistency is Key:** Eating regular meals and snacks can hel